In [ ]:
import cv2
import numpy as np
import copy
import os
import django
import pandas as pd
import matplotlib.pyplot as plt
from submissions.digits_classify import *

In [ ]:
filepath = "/home/user/Downloads/quiz.pdf"
dpi = 300

def pdf_to_images(filepath, dpi, top_percent=0.25, left_percent=0.5, crop_box=None, skip_pages=(0,1,3)):
    """
    Converts a pdf file to a list of images.
    If crop_box is not None, then the images are cropped to the specified box.
    Otherwise, the images are cropped to the top left corner of the page,
    with the width and height specified by top_percent and left_percent.

    Parameters
    ----------
    filepath : str
        The path to the pdf file.
    dpi : int
        The dpi of the images.
    top_percent : float
        The percentage of the top of the page to keep.
    left_percent : float
        The percentage of the left of the page to crop.
    crop_box : tuple
        A tuple of the form (left, top, right, bottom) that specifies the crop box.
    skip_pages : tuple
        A tuple of page numbers to skip.

    Returns
    -------
    images : list
        A list of images.
    """
    import fitz
    from PIL import Image
    images = []
    doc = fitz.open(filepath)
    for page in doc:
        if page.number % 4 in skip_pages:
            images.append(None)
            continue
        print(page.number, end="\r")
        rect = page.rect  # the page rectangle
        rect.y1 = rect.y0 + (rect.y1 - rect.y0) * top_percent
        rect.x1 = rect.x0 + (rect.x1 - rect.x0) * left_percent

        pix = page.get_pixmap(dpi=dpi, clip=rect)
        images.append(Image.frombytes(mode="RGB", size=[pix.width, pix.height], data=pix.samples))
    
    return images

quiz_imgs = pdf_to_images(filepath, dpi, top_percent=0.25, left_percent=0.5)


In [ ]:
quiz_imgs.__len__()

In [ ]:
num_pages_per_submission = 4  
if len(quiz_imgs) % num_pages_per_submission != 0:
    raise ValueError(
        f"The number of pages in the pdf is not a multiple of {num_pages_per_submission}")

# convert imgs to nested list every `num_pages_per_quiz`
# for example, if num_pages_per_quiz=2, then:
# [ [img1, img2], [img3, img4], ... ]
quizzes_img_list = [list(a) for a in zip(*[iter(quiz_imgs)] * num_pages_per_submission)]


In [ ]:
print("Creating a copy of the image list")
img_list = copy.deepcopy(quizzes_img_list)

In [ ]:
import matplotlib.pyplot as plt
print(img_list[2][2].size)
plt.imshow(img_list[2][2])

In [ ]:
for sub_idx in range(len(img_list)):
    img = np.array(img_list[sub_idx][2])
    # copy the image
    img_copy = copy.deepcopy(img)
    # convert the image to grayscale
    img_gray = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)
    # apply a threshold to the image
    ret, thresh = cv2.threshold(img_gray, 127, 255, 0)
    # find the contours in the image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # sort the contours by area
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    count = 0
    for i in range(100):
        if cv2.contourArea(contours[i]) < 100000 and cv2.contourArea(contours[i]) > 10000:
            count += 1
            # approximate the contour with a rectangle
            rect = cv2.minAreaRect(contours[i])
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            # or more concisely
            min_x, min_y = np.min(box, axis=0)
            max_x, max_y = np.max(box, axis=0)
            # pad 10 pixels on each side and crop the image
            img_crop = img[min_y-3:max_y+3, min_x-3:max_x+3]

            # plt.imshow(img_crop)
            # plt.show()

In [ ]:
# template_path = '/home/ionmich/repos/instructor_pilot/media/template.png'
# template_150 = cv2.imread(template_path, cv2.IMREAD_UNCHANGED)
# template = apply_scale_rotate(template_150, dpi//150, 0)

In [ ]:
# plt.imshow(template, cmap='gray')

In [ ]:
def remove_lines_img(
    img, 
    mode, 
    rect_kernel_size=30, 
    rect_kernel_width=1, 
    restruct_kernel_size=1, 
    do_reconstruct=False,
    dpi=150):

    if mode == "vertical":
        rect_kernel = (rect_kernel_width,rect_kernel_size)
        restruct_kernel = (restruct_kernel_size,rect_kernel_width)
    elif mode == "horizontal":
        rect_kernel = (rect_kernel_size,rect_kernel_width)
        restruct_kernel = (rect_kernel_width,restruct_kernel_size)
    else:
        raise ValueError("mode must be either 'vertical' or 'horizontal'")
    removed = img.copy()
    
    thresh = cv2.threshold(removed, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    if mode == "vertical":
        plt.imshow(removed, cmap="gray")
        plt.show()
        plt.imshow(thresh, cmap="gray")
        plt.show()
    # Remove vertical
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, rect_kernel)
    detected_lines = cv2.morphologyEx(
        thresh, 
        cv2.MORPH_OPEN, 
        vertical_kernel, 
        iterations=1)
    cnts = cv2.findContours(
        detected_lines, 
        cv2.RETR_EXTERNAL, 
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    print(f"Found {len(cnts)} lines in {mode} mode.")
    print(f"Checking if they are in the right location...")

    if cnts:
        if mode == "vertical":
            x_left = min(c.mean(axis=0)[0,0] for c in cnts)
            x_right = max(c.mean(axis=0)[0,0] for c in cnts)
            avg_x_distance = (x_right - x_left) / UFID_LENGTH
            low, high = x_left, x_right
        elif mode == "horizontal":
            x_top = min(c.mean(axis=0)[0,1] for c in cnts)
            x_bottom = max(c.mean(axis=0)[0,1] for c in cnts)
            low, high = x_top, x_bottom
    else:
        return img, (None, None)

    count_lines = 0
    for c in cnts:
        if mode=="vertical":
            x_loc = c.mean(axis=0)[0,0]
            # if x_loc is approximately multiple of avg_x_distance, then it's a valid line
            if (abs((x_loc - x_left) % avg_x_distance)>2*(dpi//150)
            and abs((x_loc - x_left) % avg_x_distance - avg_x_distance)>2*(dpi//150)
            ):
                print("Found vertical line, but won't remove because it's not a multiple of avg_x_distance away from x_left")
                continue
        count_lines += 1
        cv2.drawContours(removed, [c], -1, (255,255,255), 2*dpi//150)
    
    print(f"Overall, {count_lines} lines were removed in {mode} mode.")

    plt.imshow(removed, cmap="gray")
    plt.show()
    # Repair image
    if do_reconstruct:
        repair_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        removed = 255 - removed
        dilate = cv2.dilate(removed, repair_kernel, iterations=3)
        pre_result = cv2.bitwise_and(dilate, thresh)

        result = cv2.morphologyEx(pre_result, cv2.MORPH_CLOSE, repair_kernel, iterations=3)
        final = cv2.bitwise_and(result, thresh)
        invert_final = 255 - final
    else:
        invert_final = removed

    return invert_final , (low, high)

def apply_scale_rotate(img, scale, angle):
    """Apply scale and rotation to image"""
    width = int(img.shape[1] * scale)
    height = int(img.shape[0] * scale)
    dim = (width, height)
    scaled_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    M = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
    rotated_img = cv2.warpAffine(scaled_img, M, (width, height))
    return rotated_img

def find_best_scale_and_angle(template, img_list, n_iter=100, skip_pages=[0,1,3], match_method=None):
    """Do a Monte Carlo search for the best scale and angle to align the template to the image"""
    import random
    # temperature
    T = 1
    if match_method is None:
        match_method = cv2.TM_SQDIFF_NORMED
    if match_method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        best_score = np.inf
    else:
        best_score = 0
    best_scale = 1.0
    best_angle = 0
    # step size for Marcov chain
    scale_sigma = 0.01
    angle_sigma = 0.01
    # start with scale and angle of 1.0 and 0
    scale = best_scale
    angle = best_angle
    score = best_score
    # prior for scale and angle is gaussian with mean 1.0 and 0
    # and variances 0.1 and 1.0
    scale_prior = lambda x: np.exp(-0.5*(x-1.0)**2/0.1**2)
    angle_prior = lambda x: np.exp(-0.5*x**2/1.0**2)
    prior = lambda x,y: scale_prior(x)*angle_prior(y)
    for i in range(n_iter):
        # scale and angle search
        proposed_scale = scale + random.gauss(0, scale_sigma)
        propose_angle =  angle + random.gauss(0, angle_sigma)
        print("proposed: scale", proposed_scale, "angle", propose_angle, end="\r")
        # scale the template
        template_ = apply_scale_rotate(template, proposed_scale, propose_angle)
        template_rgb, template_luminance = template_[:,:,:3], template_[:,:,3]
        
        # randomly choose an image to match to
        sub_idx = random.randint(0, len(img_list)-1)
        # select page_idx randomly from 0 to 3 except for skip_pages
        page_idx = random.choice([i for i in range(4) if i not in skip_pages])
        img_rgb = np.array(img_list[sub_idx][page_idx])
        # image_to_match = cv2.erode(img_rgb, None, iterations=1)
        # template_rgb_ = cv2.erode(template_rgb, None, iterations=1)
        # copy
        image_to_match = img_rgb.copy()
        template_rgb_ = template_rgb.copy()
        result = cv2.matchTemplate(image_to_match, template_rgb_, match_method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        if match_method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
            # implement the Metropolis-Hastings acceptance criterion
            # accept if the new score is better than the old score
            # or if the new score is worse than the old score, accept with probability exp(-(new-old)/T)
            prior_ratio = prior(proposed_scale, propose_angle) / prior(scale, angle)
            if min_val < best_score or random.random() < prior_ratio * np.exp(-(min_val-best_score)/T):
                angle = propose_angle
                scale = proposed_scale
            if min_val < best_score:
                best_score = min_val
                best_scale = proposed_scale
                best_angle = propose_angle
        else:
            if max_val > best_score or random.random() < prior_ratio * np.exp((max_val - best_score)/T):
                angle = propose_angle
                scale = proposed_scale
            if max_val > best_score:
                best_score = max_val
                best_scale = proposed_scale
                best_angle = propose_angle
        # print and overwrite in the same line to show progress
    print("best: scale", best_scale, "angle", best_angle, "score", best_score)
    best_template = apply_scale_rotate(template, best_scale, best_angle)
    return best_template, best_scale, best_angle

def get_possible_boundaries(padding_px):
    sub_boundaries = {}
    for sub_idx in range(len(img_list)):
        sub_boundaries[sub_idx] = []
        img = np.array(img_list[sub_idx][2])
        # copy the image
        img_copy = img
        # convert the image to grayscale
        img_gray = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)
        # apply a threshold to the image
        ret, thresh = cv2.threshold(img_gray, 127, 255, 0)
        # find the contours in the image
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # sort the contours by area
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        count = 0
        for i in range(100):
            if cv2.contourArea(contours[i]) < 100000 and cv2.contourArea(contours[i]) > 10000:
                count += 1
                # approximate the contour with a rectangle
                rect = cv2.minAreaRect(contours[i])
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                # or more concisely
                min_x, min_y = np.min(box, axis=0)
                max_x, max_y = np.max(box, axis=0)
                # pad 
                sub_boundaries[sub_idx].append((
                    max(0, min_x-padding_px),
                    max(0, min_y-padding_px),
                    min(img.shape[1], max_x+padding_px),
                    min(img.shape[0], max_y+padding_px)
                ))
        
    return sub_boundaries

def new_main(img_list):
    # get possible boundaries
    padding_px = round(3 * dpi / 150)
    sub_boundaries = get_possible_boundaries(padding_px)
    
    for sub_idx in range(len(img_list)):
        for page_idx in range(len(img_list[sub_idx])):
            if page_idx != 2:
                continue
            # now we have the boundaries so we can crop the image
            # withouth the need of the template!
            img_rgb = np.array(img_list[sub_idx][page_idx])
            img = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
            # try each possible boundary
            for boundary in sub_boundaries[sub_idx]:
                try:
                    img_ = img[boundary[1]:boundary[3], boundary[0]:boundary[2]]
                    plt.imshow(img_, cmap="gray")
                    plt.show()

                    img_no_v, (x_left, x_right) = remove_lines_img(
                        img_, 
                        mode="vertical", 
                        rect_kernel_size=30*dpi//150, 
                        rect_kernel_width=1, 
                        do_reconstruct=False
                        )

                    img_no_h, _ = remove_lines_img(
                        img_, 
                        mode="horizontal", 
                        rect_kernel_size=70*dpi//150, 
                        rect_kernel_width=1, 
                        do_reconstruct=False
                        )
                    img_nolines = cv2.bitwise_or(img_no_v, img_no_h)
                    plt.imshow(img_nolines, cmap="gray")
                    plt.show()
                except Exception as e:
                    print(e)
                    plt.imshow(img_, cmap="gray")
                    plt.show()
                    continue
new_main(img_list)


In [ ]:
digit_imgs = get_all_digits_new(
    img_list,
    dpi=300,
    pages_to_skip=(0,1,3),
)
# note that the digit_imgs is a dict with (idx_submission, idx_page) as key 
# and an image (np.array) as value

In [ ]:
for i in range(len(img_list)):
    # get the images of digits in page 3
    imgs = digit_imgs[(i, 2)]
    
    # show the output images in cmap gray, with 8 images per row
    fig, ax = plt.subplots(1, len(imgs), figsize=(20, 20))
    for j in range(len(imgs)):
        ax[j].imshow(imgs[j], cmap="gray")
        ax[j].axis("off")
    plt.show()

In [ ]:
template = cv2.imread("media/template.png", cv2.IMREAD_GRAYSCALE)
# pad 100 pixels to the template
template = np.pad(template, 100, mode="constant", constant_values=255)
# rotate the template by 20 degrees
template = apply_scale_rotate(template, 10, 20)

# plot
plt.imshow(template, cmap="gray")
plt.show()

In [ ]:
# calculate the hough transform
# and plot the result
src = template

dst = cv2.Canny(src, 50, 200, None, 3)

# Copy edges to the images that will display the results in BGR
cdst = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)


linesP = cv2.HoughLinesP(dst, 1, np.pi / 180, 10, None, 20, 10)

if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv2.line(cdst, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)

# or with plt
plt.imshow(src, cmap="gray")
plt.show()
plt.imshow(cdst, cmap="gray")
plt.show()